Pandas array

	- Task: TaskA or TaskB
	- Val1-Val4
	- If Task A, Score1=Val2-Val1, Score2=Val4-Val3
	- If Task B, Score1=f(Val1,Val2), Score2=f(Val3,Val4)
    
F(Val1,Val2) = np.normppf(Val2)-np.normppf(Val1), BUT if either val == 0, fudge it to 0.01 and if either value == 1, fudge it to 0.99

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [7]:
# Two variants of even just how to create the data
def create_data_old(n=100):
    # Old method of randomness
    tasks=np.random.choice(['TaskA','TaskB'],n)
    vals=np.random.random_sample((n,4))
    nbad=int(n/20)
    for i in range(4): # Put in some "bad" values
        vals[np.random.choice(vals.shape[0],nbad,False)]=0
        vals[np.random.choice(vals.shape[0],nbad,False)]=1
    df=pd.DataFrame(vals,columns=['val1','val2','val3','val4'])
    df['task']=tasks
    return df

def create_data(n=100):
    rng=np.random.default_rng()
    tasks=rng.choice(['TaskA','TaskB'],n)
    vals=rng.random((n,4))
    nbad=int(n/20)
    def flagbad(d,n):
        rng=np.random.default_rng()
        d[rng.choice(d.shape[0],n,False)]=0
        d[rng.choice(d.shape[0],n,False)]=1
    np.apply_along_axis(flagbad,0,vals,nbad)
    return pd.DataFrame({'task':tasks,
                 'val1':vals[:,0],'val2':vals[:,1],'val3':vals[:,2],'val4':vals[:,3]})

In [9]:
# Let's see how fast even this just is
%timeit create_data_old(100000)
%timeit create_data(100000)

10.3 ms ± 144 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.51 ms ± 56.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


First up, some very simple, standard ways of looping...

In [10]:
def compute_score_loop_iloc_1(data):
    df=data.copy()
    # Add the needed columns and set to 0
    df['score1']=0.0
    df['score2']=0.0
    # Since we're going to use iloc, get the column numbers for these ahead of time
    v1col=df.columns.get_loc('val1')
    v2col=df.columns.get_loc('val2')
    v3col=df.columns.get_loc('val3')
    v4col=df.columns.get_loc('val4')
    s1col=df.columns.get_loc('score1')
    s2col=df.columns.get_loc('score2')
    tcol=df.columns.get_loc('task')
    for i in range(df.shape[0]): # Loop over each element
        if df.iloc[i,tcol]=='TaskA':
            df.iloc[i,s1col]=df.iloc[i,v2col]-df.iloc[i,v1col]
            df.iloc[i,s2col]=df.iloc[i,v4col]-df.iloc[i,v3col]
        else:
            v1=df.iloc[i,v1col]
            v2=df.iloc[i,v2col]
            v3=df.iloc[i,v3col]
            v4=df.iloc[i,v4col]
            if v1==0.0:
                v1=0.01
            elif v1==1.0:
                v1=0.99
            if v2==0.0:
                v2=0.01
            elif v2==1.0:
                v2=0.99
            if v3==0.0:
                v3=0.01
            elif v3==1.0:
                v3=0.99
            if v4==0.0:
                v4=0.01
            elif v4==1.0:
                v4=0.99
            df.iloc[i,s1col]=norm.ppf(v2)-norm.ppf(v1)
            df.iloc[i,s2col]=norm.ppf(v4)-norm.ppf(v3)
    return df


def compute_score_loop_iloc_2(data):
    df=data.copy()
    # Add the needed columns and set to 0
    df['score1']=0.0
    df['score2']=0.0
    # Since we're going to use iloc, get the column numbers for these ahead of time
    v1col=df.columns.get_loc('val1')
    v2col=df.columns.get_loc('val2')
    v3col=df.columns.get_loc('val3')
    v4col=df.columns.get_loc('val4')
    s1col=df.columns.get_loc('score1')
    s2col=df.columns.get_loc('score2')
    tcol=df.columns.get_loc('task')
    for i in range(df.shape[0]): # Loop over each element
        if df.iloc[i,tcol]=='TaskA':
            df.iloc[i,s1col]=df.iloc[i,v2col]-df.iloc[i,v1col]
            df.iloc[i,s2col]=df.iloc[i,v4col]-df.iloc[i,v3col]
        else:
            # Use numpy to do our clipping
            v1=np.clip(df.iloc[i,v1col],0.01, 0.99)
            v2=np.clip(df.iloc[i,v2col],0.01, 0.99)
            v3=np.clip(df.iloc[i,v3col],0.01, 0.99)
            v4=np.clip(df.iloc[i,v4col],0.01, 0.99)
            df.iloc[i,s1col]=norm.ppf(v2)-norm.ppf(v1)
            df.iloc[i,s2col]=norm.ppf(v4)-norm.ppf(v3)
    return df

def compute_score_loop_loc(data):
    df=data.copy()
    # Add the needed columns and set to 0
    df['score1']=0.0
    df['score2']=0.0

    for i in df.index: # Loop over each element
        if df.loc[i,'task']=='TaskA':
            df.loc[i,'score1']=df.loc[i,'val2'] - df.loc[i,'val1']
            df.loc[i,'score2']=df.loc[i,'val4'] - df.loc[i,'val3']
        else:
            # Use numpy to do our clipping
            v1=np.clip(df.loc[i,'val1'],0.01, 0.99)
            v2=np.clip(df.loc[i,'val2'],0.01, 0.99)
            v3=np.clip(df.loc[i,'val3'],0.01, 0.99)
            v4=np.clip(df.loc[i,'val4'],0.01, 0.99)
            df.loc[i,'score1']=norm.ppf(v2)-norm.ppf(v1)
            df.loc[i,'score2']=norm.ppf(v4)-norm.ppf(v3)
    return df

In [11]:
# Let's run some timing
data=create_data(1000)
print('compute_score_loop_iloc_1')
%timeit compute_score_loop_iloc_1(data)
print('compute_score_loop_iloc_2')
%timeit compute_score_loop_iloc_2(data)
print('compute_score_loop_loc')
%timeit compute_score_loop_loc(data)


compute_score_loop_iloc_1
143 ms ± 1.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
compute_score_loop_iloc_2
150 ms ± 4.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
compute_score_loop_loc
170 ms ± 4.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
def compute_score_pandas_group(data):
    # Note, this will change the order to be sorted by groups.
    #  can make it return taska.append(taskb).sort_index() 

    df=data.copy()
    # Add the needed columns and set to 0
    df['score1']=0.0
    df['score2']=0.0
    # Split this out into our two tasks
    grp=data.groupby('task')
    taska=grp.get_group('TaskA').copy()
    taskb=grp.get_group('TaskB').copy()
    taska['score1']=taska['val2']-taska['val1']
    taska['score2']=taska['val4']-taska['val3']
    taskb['score1']=norm.ppf(np.clip(taskb['val2'],0.01,0.99))-norm.ppf(np.clip(taskb['val1'],0.01,0.99))
    taskb['score2']=norm.ppf(np.clip(taskb['val4'],0.01,0.99))-norm.ppf(np.clip(taskb['val3'],0.01,0.99))
    return pd.concat([taska,taskb],ignore_index=True)

def compute_score_pandas_logicalindex(data):    
    df=data.copy()    # Add the needed columns and set to 0    
    df['score1']=0.0    
    df['score2']=0.0    # Split this out into our two tasks    
    taska=df['task']=='TaskA'
    taskb=df['task']=='TaskB'
    
    df.loc[taska,'score1']=df.loc[taska,'val2']-df.loc[taska,'val1']    
    df.loc[taska,'score2']=df.loc[taska,'val4']-df.loc[taska,'val3']    
    df.loc[taskb,'score1']=norm.ppf(np.clip(df.loc[taskb,'val2'],0.01,0.99))-norm.ppf(np.clip(df.loc[taskb,'val1'],0.01,0.99))    
    df.loc[taskb,'score2']=norm.ppf(np.clip(df.loc[taskb,'val4'],0.01,0.99))-norm.ppf(np.clip(df.loc[taskb,'val3'],0.01,0.99))    
    return df


In [15]:
print ('Pandas logical by group')
%timeit compute_score_pandas_group(data)

print ('Pandas logical indexing')
%timeit compute_score_pandas_logicalindex(data)

Pandas logical by group
1.43 ms ± 19.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Pandas logical indexing
1.71 ms ± 5.37 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
